In [2]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/xgabora/Club-Football-Match-Data-2000-2025/refs/heads/main/data/Matches.csv")
df = df[df["Division"] == 'E0']
df

C:\Users\sunc2\AppData\Local\Temp\ipykernel_21064\3752613371.py:1: DtypeWarning: Columns (0: MatchTime) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://raw.githubusercontent.com/xgabora/Club-Football-Match-Data-2000-2025/refs/heads/main/data/Matches.csv")


,Division,MatchDate,MatchTime,HomeTeam,AwayTeam,HomeElo,AwayElo,Form3Home,Form5Home,Form3Away,...,MaxUnder25,HandiSize,HandiHome,HandiAway,C_LTH,C_LTA,C_VHD,C_VAD,C_HTB,C_PHB
154,E0,2000-08-19,NaN,Charlton,Man City,1608.77,1579.99,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.1396,0.0346,0.2085,0.0100,0.2939,0.3134
155,E0,2000-08-19,NaN,Chelsea,West Ham,1800.17,1681.36,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0702,0.0742,0.0099,0.0823,0.0387,0.7247
156,E0,2000-08-19,NaN,Coventry,Middlesbrough,1635.61,1679.18,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.8114,0.0316,0.0103,0.0731,0.0319,0.0417
157,E0,2000-08-19,NaN,Derby,Southampton,1636.08,1630.02,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0272,0.0390,0.0467,0.0100,0.0961,0.7810
158,E0,2000-08-19,NaN,Leeds,Everton,1782.55,1685.55,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.7772,0.0994,0.0140,0.0429,0.0119,0.0546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230506,E0,2025-05-25,16:00:00,Newcastle,Everton,1882.33,1776.20,4.0,7.0,7.0,...,2.65,-1.5,1.93,1.93,0.0276,0.0613,0.2894,0.0100,0.5967,0.0150
230507,E0,2025-05-25,16:00:00,Nott'm Forest,Chelsea,1799.76,1880.42,5.0,8.0,6.0,...,2.26,0.3,1.93,1.93,0.1250,0.7378,0.0139,0.0637,0.0291,0.0305
230508,E0,2025-05-25,16:00:00,Southampton,Arsenal,1557.66,1985.45,1.0,2.0,4.0,...,2.80,1.5,2.03,1.83,0.0105,0.0098,0.0098,0.9324,0.0276,0.0098
230509,E0,2025-05-25,16:00:00,Tottenham,Brighton,1784.55,1797.83,1.0,1.0,7.0,...,3.55,0.8,2.00,1.85,0.2559,0.0320,0.0136,0.4660,0.1900,0.0425


In [4]:
def eloPred(eloDiff, HomeElo, AwayElo):
    avgElo = (HomeElo + AwayElo) / 2
    if abs(eloDiff / avgElo) <= 0.0125: 
        return (eloDiff, 'D')
    elif eloDiff > 0:
        return (eloDiff, 'H')
    else: 
        return(eloDiff, 'A') 

In [5]:
df["EloDiff"] = df["HomeElo"] - df["AwayElo"]
df["EloPred"] = df["EloDiff"].apply(lambda x: 'H' if x > 0 else ('A' if x < 0 else 'D'))

df['EloPred (Draw Zone)'] = df.apply(lambda row: eloPred(row['EloDiff'], row['HomeElo'], row['AwayElo'])[1], axis=1)
acc = (df['EloPred'] == df['FTResult']).mean()
acc_dz = (df['EloPred (Draw Zone)'] == df['FTResult']).mean()

print(f"Elo prediction accuracy (no draw zone): {acc:.2%}")
print(f"Elo prediction accuracy (with 1.25% draw zone): {acc_dz:.2%}")

Elo prediction accuracy (no draw zone): 51.51%
Elo prediction accuracy (with 1.25% draw zone): 50.63%


In [6]:
# fig, ax = plt.subplots(figsize=(8, 5))
# df.boxplot(column='EloDiff', by='FTResult', ax=ax, showmeans=True)
# means = df.groupby('FTResult')['EloDiff'].mean()
# for i, (label, val) in enumerate(means.items(), 1): 
#     ax.text(i, val, f'{val:.1f}', ha='center', va='bottom', fontweight='bold')
# ax.set_title("Elo Difference by Match Result")
# plt.suptitle('')
# ax.set_ylabel('Home Elo - Away Elo')
# plt.show()

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [8]:
features = ['HomeElo', 'AwayElo', 'EloDiff', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'C_LTH', 'C_LTA', 'C_VHD', 'C_VAD', 'C_HTB', 'C_PHB']
target = 'FTResult'

model_df = df[features + [target]].dropna()
X = model_df[features]
y = model_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

# logistic regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_scaled, y_train)
lr_acc = accuracy_score(y_test, lr.predict(X_test_scaled))
print(f"Logistic Regression Accuracy: {lr_acc:.2%}")

# random forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf.acc = accuracy_score(y_test, rf.predict(X_test))
print(f"Random Forest Accuracy: {rf.acc:.2%}")

Logistic Regression Accuracy: 53.72%
Random Forest Accuracy: 52.71%
